In [4]:
from ortools.linear_solver import pywraplp
import random
import pandas as pd


# Função que recolhe as variáveis do utilizador... tem valores padrão pré-definidos no caso do utilizador não introduzir valores
def get_input(prompt, default):
    user_input = input(f"{prompt} [{default}]: ")  # Mostra o valor padrão
    return int(user_input) if user_input else default  # Usa o valor padrão se o input estiver vazio

#Função que executa o solver com os parâmetros dados
def executar_solver(S, D, H, P, C, Colaboradores, Projectos):
    # Inicializando o solver SCIP
    horario = pywraplp.Solver.CreateSolver('SCIP')
    
    # Variáveis de decisão: x[s, d, h, p, c] = 1 se o colaborador c participa na reunião do projeto p no slot (sala s, dia d, hora h)
    x = {}
    for s in range(S):
        for d in range(D):
            for h in range(H):
                for p in range(P):
                    for c in range(1, C + 1):
                        x[s, d, h, p, c] = horario.BoolVar(f'x[{s},{d},{h},{p},{c}]')
    
    # Restrições: o líder do projeto deve estar presente durante o número de slots alocado ao projeto
    for p in range(P):
        lider = Projectos[p][1][0]
        horario.Add(
            sum(x[s, d, h, p, lider] for s in range(S) for d in range(D) for h in range(H)) == Projectos[p][0]
        )
    
    # Restrição: o colaborador só pode estar em slots que ele está disponível
    for s in range(S):
        for d in range(D):
            for h in range(H):
                for p in range(P):
                    for c in range(1, C + 1):
                        if (d, h) not in Colaboradores[c-1]:
                            horario.Add(x[s, d, h, p, c] == 0)

    # Restrição: colaboradores que não fazem parte do projeto não podem participar das reuniões do projeto
    for s in range(S):
        for d in range(D):
            for h in range(H):
                for p in range(P):
                    for c in range(1, C + 1):
                        if c not in Projectos[p][1]:
                            horario.Add(x[s, d, h, p, c] == 0)

    # Restrição: uma sala só pode ter uma reunião por vez
    for s in range(S):
        for d in range(D):
            for h in range(H):
                horario.Add(sum(x[s, d, h, p, Projectos[p][1][0]] for p in range(P)) <= 1)

    # Restrição: um colaborador não pode estar em mais de uma reunião ao mesmo tempo
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in Projectos[p][1]:
                    horario.Add(sum(x[s, d, h, p, c] for s in range(S)) <= 1)

    # Restrição de quorum: metade ou mais dos colaboradores de um projeto devem estar presentes, incluindo o líder
    for s in range(S):
        for d in range(D):
            for h in range(H):
                for p in range(P):
                    quorum = len(Projectos[p][1]) // 2 + 1  # 50% ou mais com o líder
                    horario.Add(sum(x[s, d, h, p, c] for c in Projectos[p][1]) >= quorum * x[s, d, h, p, Projectos[p][1][0]])

    # Função objetivo: maximizar o número total de slots alocados a reuniões
    horario.Maximize(
        sum(x[s, d, h, p, c] for s in range(S) for d in range(D) for h in range(H) for p in range(P) for c in range(1, C + 1))
    )

    # Resolver o problema
    status = horario.Solve()

    # Verificar a solução e preparar a tabela de resultados
    schedule_data = []

    if status == pywraplp.Solver.OPTIMAL:
        for p in range(P):
            for s in range(S):
                for d in range(D):
                    for h in range(H):
                        if round(x[s, d, h, p, Projectos[p][1][0]].solution_value()) == 1:
                            participants = [c for c in Projectos[p][1] if round(x[s, d, h, p, c].solution_value()) == 1]
                            schedule_data.append([p + 1, s + 1, f"{d + 1},{h + 1}", participants])
    else:
        schedule_data.append([None, None, None, "Nenhuma solução ótima encontrada."])

    df_schedule = pd.DataFrame(schedule_data, columns=["Projeto", "Sala", "Dia_Hora", "Participantes"])

    def style_schedule(df):
        return df.style.set_table_styles(
            [{'selector': 'tr:nth-child(even)',
              'props': [('background-color', '#f2f2f2')]},   # Cor alternada para linhas pares
             {'selector': 'th, td',
              'props': [('border', '1px solid black')]}      # Bordas em volta das células
            ]
        ).set_properties(**{'text-align': 'center'})          # Alinhar o texto no centro

    # Aplicar estilo e exibir diretamente no notebook
    styled_table = style_schedule(df_schedule)

    # Exibir a tabela estilizada no Jupyter Notebook sem o índice
    display(styled_table.hide(axis='index'))

# Testes automáticos para validar o funcionamento
def testes_automaticos():
    # Teste 1: Parâmetros pequenos para testar a lógica básica
    S, D, H, P, C = 2, 2, 4, 2, 6
    Slots = [(d, h) for d in range(D) for h in range(H)]
    Colaboradores = [random.sample(Slots, 4) for _ in range(1, C + 1)]
    Colabs = set(range(1, C + 1))
    Projectos = []
    for _ in range(P):
        available_team_size = min(3, len(Colabs))
        team = random.sample(list(Colabs), available_team_size)
        Projectos.append((random.randint(1, 2), team))
        Colabs = Colabs - set(team)
    print(f"Teste 1: {S} salas, {D} dias, {H} slots por dia, {P} projetos, {C} colaboradores.")
    executar_solver(S, D, H, P, C, Colaboradores, Projectos)

    # Teste 2: Parâmetros maiores para testar o escalonamento
    S, D, H, P, C = 5, 5, 8, 4, 15
    Slots = [(d, h) for d in range(D) for h in range(H)]
    Colaboradores = [random.sample(Slots, 10) for _ in range(1, C + 1)]
    Colabs = set(range(1, C + 1))
    Projectos = []
    for _ in range(P):
        available_team_size = min(5, len(Colabs))
        team = random.sample(list(Colabs), available_team_size)
        Projectos.append((random.randint(2, 3), team))
        Colabs = Colabs - set(team)
    print(f"Teste 2: {S} salas, {D} dias, {H} slots por dia, {P} projetos, {C} colaboradores.")
    executar_solver(S, D, H, P, C, Colaboradores, Projectos)
    
    
     # Teste 3: Elevado número de colaboradores e salas para testar escalabilidade
    S, D, H, P, C = 10, 5, 8, 6, 50
    Slots = [(d, h) for d in range(D) for h in range(H)]
    Colaboradores = [random.sample(Slots, 15) for _ in range(1, C + 1)]
    Colabs = set(range(1, C + 1))
    Projectos = []
    for _ in range(P):
        available_team_size = min(5, len(Colabs))  # Equipas maiores
        team = random.sample(list(Colabs), available_team_size)
        Projectos.append((random.randint(2, 4), team))
        Colabs = Colabs - set(team)
    print(f"Teste 3: {S} salas, {D} dias, {H} slots por dia, {P} projetos, {C} colaboradores.")
    executar_solver(S, D, H, P, C, Colaboradores, Projectos)
    
        # Teste 4: Mais projetos do que o número de colaboradores disponíveis
    S, D, H, P, C = 4, 3, 5, 6, 10
    Slots = [(d, h) for d in range(D) for h in range(H)]
    Colaboradores = [random.sample(Slots, 5) for _ in range(1, C + 1)]
    Colabs = set(range(1, C + 1))
    Projectos = []
    for _ in range(P):
        available_team_size = min(3, len(Colabs))  # Equipes menores
        team = random.sample(list(Colabs), available_team_size)
        Projectos.append((random.randint(1, 2), team))
        Colabs = Colabs - set(team)
    print(f"Teste 4: {S} salas, {D} dias, {H} slots por dia, {P} projetos, {C} colaboradores.")
    executar_solver(S, D, H, P, C, Colaboradores, Projectos)

    # Teste 5: Apenas um projeto com muitos colaboradores
    S, D, H, P, C = 3, 3, 6, 1, 20
    Slots = [(d, h) for d in range(D) for h in range(H)]
    Colaboradores = [random.sample(Slots, 8) for _ in range(1, C + 1)]
    Colabs = set(range(1, C + 1))
    Projectos = []
    for _ in range(P):
        available_team_size = len(Colabs)  # Todos os colaboradores no único projeto
        team = random.sample(list(Colabs), available_team_size)
        Projectos.append((random.randint(3, 4), team))
        Colabs = Colabs - set(team)
    print(f"Teste 5: {S} salas, {D} dias, {H} slots por dia, {P} projetos, {C} colaboradores.")
    executar_solver(S, D, H, P, C, Colaboradores, Projectos)

# Função para rodar todos os testes
testes_automaticos()

    
    

    

Teste 1: 2 salas, 2 dias, 4 slots por dia, 2 projetos, 6 colaboradores.


Projeto,Sala,Dia_Hora,Participantes
1,1,"1,4","[3, 5]"
2,1,"1,2","[2, 1]"
2,2,"2,4","[2, 1]"


Teste 2: 5 salas, 5 dias, 8 slots por dia, 4 projetos, 15 colaboradores.


IndexError: list index out of range